In [1]:


from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

In [3]:
import scanner_params

scanner_params.save_scanner_params_as_xml_from_ib(ib)
scanner_params.create_md_documentaion_from_scan_params_xml()

Scanner parameters have been saved to ib_scan_params.xml
Documentation has been saved to ib_scanner_documentation.md


In [3]:
import pandas as pd


class StockbotScanner:
    def __init__(self, ib):
        self.ib = ib
        self.scan_results_df = None

    def scan(self, 
            scan_code:str ='TOP_PERC_GAIN',
            price: tuple[float, float] = (1, 100),
            volume: int = 100_000,
            change_perc: float = 4,
            market_cap: tuple[float, float] = (100, 10000), # value is in millions
            location='STK.US.MAJOR',
            ):
        
        # set tags for scanner
        tags = [
            TagValue('priceAbove', price[0]),      
            TagValue('priceBelow', price[1]),
            TagValue('volumeAbove', volume),   
            TagValue('changePercAbove', change_perc),
            TagValue('marketCapAbove1e6', market_cap[0]), # value is in millions
            TagValue('marketCapBelow1e6', market_cap[1]) # value is in millions
        ]
        
        # Request scanner data
        sub = ScannerSubscription(
            instrument='STK',
            locationCode=location,
            scanCode=scan_code
        )

        print(f'Scanning {location} for {scan_code} ..')
        print(f'price: {price}, volume: {volume}, changePerc: {change_perc}, marketCap: {market_cap}')

        data = self.ib.reqScannerData(sub, [], tags)
        print(f'{location} .. len(data): {len(data)}')
        return data
    
    def multiscan(self, 
            scan_code: str = 'TOP_PERC_GAIN',
            price: tuple[float, float] = (1, 100),
            volume: int = 100_000,
            change_perc: float = 4,
            market_cap: float = 100):
        
        market_caps = [(market_cap, market_cap*10), (market_cap*10, market_cap*100), (market_cap*100, market_cap*1000)]
        rows = []

        for cap in market_caps:
            scanData = self.scan(scan_code, price, volume, change_perc, cap)
            cap_str = f'{cap[0]}M-{cap[1]}M'
            
            for data in scanData:
                contract = data.contractDetails.contract
                row = {
                    'rank': data.rank,
                    'market_cap_range': cap,
                    'symbol': contract.symbol,
                    'exchange': contract.exchange,
                    'currency': contract.currency,
                    'primaryExchange': contract.primaryExchange,
                }
                rows.append(row)
                
            self.ib.sleep(60)
        
        # Create DataFrame and set column order
        df = pd.DataFrame(rows)
        
        # Define column order with rank and market_cap_range first
        columns = ['rank', 'market_cap_range', 'symbol', 'exchange', 'currency', 'primaryExchange']
        
        # Reorder columns and sort by rank
        df = df[columns].sort_values('rank')
        
        # Store DataFrame in instance variable and return it
        self.scan_results_df = df
        return df



stockbot = StockbotScanner(ib)
stockbot.multiscan(scan_code='TOP_PERC_GAIN', price=(1, 100), volume=100_000, change_perc=4, market_cap=100)
stockbot.scan_results_df

Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (100, 1000)
STK.US.MAJOR .. len(data): 50
Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (1000, 10000)
STK.US.MAJOR .. len(data): 50
Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (10000, 100000)
STK.US.MAJOR .. len(data): 7


,rank,market_cap_range,symbol,exchange,currency,primaryExchange
0,0,"(100, 1000)",QSI,SMART,USD,
100,0,"(10000, 100000)",FLEX,SMART,USD,
50,0,"(1000, 10000)",KC,SMART,USD,
1,1,"(100, 1000)",QUBT,SMART,USD,
101,1,"(10000, 100000)",CSGP,SMART,USD,
...,...,...,...,...,...,...
47,47,"(100, 1000)",SVRA,SMART,USD,
98,48,"(1000, 10000)",CON,SMART,USD,
48,48,"(100, 1000)",AMCX,SMART,USD,
99,49,"(1000, 10000)",ARVN,SMART,USD,


In [3]:
stockbot.scan_results_df.head(50)['rank', 'market_cap_range', 'symbol']

AttributeError: 'NoneType' object has no attribute 'head'

In [38]:
sub = ScannerSubscription(
    instrument='STK',
    # locationCode='STK.US.MAJOR',
    locationCode='STK.US.MAJOR',
    scanCode='TOP_PERC_GAIN')

tagValues = [
    TagValue("changePercAbove", 20),
    TagValue('priceAbove', 5),
    TagValue('priceBelow', 50)]

# the tagValues are given as 3rd argument; the 2nd argument must always be an empty list
# (IB has not documented the 2nd argument and it's not clear what it does)
scanData = ib.reqScannerData(sub, [], tagValues)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

['SMCX', 'XCUR', 'SMCI', 'SYM', 'VRM', 'BTCT', 'CHSN', 'AI', 'DXYZ', 'DALN']


In [4]:
from dataclasses import dataclass
from typing import Optional, Union, Dict
from datetime import datetime
import pandas as pd

def parse_xml_value(ratio_element) -> Union[float, str, datetime]:
    """Extract value from a Ratio XML element, handling different data types"""
    field_type = ratio_element.get('Type', 'N')  # Default to numeric if no type specified
    
    # Get the raw value (either from text or nested Value element)
    if ratio_element.text and ratio_element.text.strip():
        raw_value = ratio_element.text.strip()
    else:
        value_elem = ratio_element.find('.//Value')
        if value_elem is not None and value_elem.text:
            raw_value = value_elem.text.strip()
        else:
            return 0.0 if field_type == 'N' else ''
    
    # Parse based on type
    try:
        if field_type == 'N':  # Numeric
            return float(raw_value)
        elif field_type == 'D':  # Date
            return raw_value  # Keep as string for now
        else:  # Default to string for unknown types
            return raw_value
    except ValueError:
        return 0.0 if field_type == 'N' else raw_value

def get_ratio_value(data: Dict[str, Union[float, str]], field: str, default: float = 0.0) -> float:
    """Safely get numeric value from data dictionary"""
    try:
        value = data.get(field, default)
        return float(value) if value != '' else default
    except (ValueError, TypeError):
        return default

@dataclass
class ForecastMetrics:
    """Forecast data from analyst estimates"""
    consensus_recommendation: float  # ConsRecom
    target_price: float             # TargetPrice
    projected_growth_rate: float    # ProjIGrowthRate
    projected_pe: float             # ProjPE
    projected_sales: float          # ProjSales
    projected_sales_growth: float   # ProjSalesQ
    projected_eps: float            # ProjEPS
    projected_eps_q: float          # ProjEPSQ
    projected_profit: float         # ProjProfit
    projected_operating_margin: float  # ProjOPS

@dataclass
class StockInfo:
    # Timestamp and Metadata
    latest_available_date: str
    price_currency: str
    reporting_currency: str
    exchange_rate: float
    
    # Price and Volume Metrics
    current_price: float
    high_52week: float
    low_52week: float
    pricing_date: str
    volume_10day_avg: float
    enterprise_value: float
    
    # Income Statement Metrics
    market_cap: float
    revenue_ttm: float
    ebitda: float
    net_income_ttm: float
    
    # Per Share Metrics
    eps_ttm: float
    revenue_per_share: float
    book_value_per_share: float
    cash_per_share: float
    cash_flow_per_share: float
    dividend_per_share: float
    
    # Margin Metrics
    gross_margin: float
    operating_margin: float
    net_profit_margin: float
    
    # Growth Metrics
    revenue_growth_rate: float
    eps_growth_rate: float
    
    # Valuation Metrics
    pe_ratio: float
    price_to_book: float
    price_to_sales: float
    
    # Company Information
    employee_count: int
    
    # Forecast Data
    forecast: Optional[ForecastMetrics] = None
    
    # Computed Metrics
    price_to_10day_avg: float = 0.0
    volume_vs_10day_avg_pct: float = 0.0
    distance_from_52wk_high_pct: float = 0.0
    distance_from_52wk_low_pct: float = 0.0

    def compute_derived_metrics(self, current_volume: float = 0):
        """Compute additional metrics based on available data"""
        if self.current_price and self.high_52week:
            self.distance_from_52wk_high_pct = ((self.high_52week - self.current_price) / self.high_52week) * 100
            
        if self.current_price and self.low_52week:
            self.distance_from_52wk_low_pct = ((self.current_price - self.low_52week) / self.low_52week) * 100
            
        if current_volume and self.volume_10day_avg:
            self.volume_vs_10day_avg_pct = ((current_volume - self.volume_10day_avg) / self.volume_10day_avg) * 100

def get_stock_info(ib, ticker: str, current_volume: float = 0) -> StockInfo:
    """Retrieve comprehensive stock information including all available ratios."""
    contract = Stock(ticker, 'SMART', 'USD')
    details = ib.reqContractDetails(contract)
    if not details:
        raise ValueError(f"No contract details found for {ticker}")
    
    fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
    root = ET.fromstring(fundamental_data)
    
    # Get root level attributes
    ratios = root.find('.//Ratios')
    price_currency = ratios.get('PriceCurrency', 'USD')
    reporting_currency = ratios.get('ReportingCurrency', 'USD')
    exchange_rate = float(ratios.get('ExchangeRate', '1.0'))
    latest_date = ratios.get('LatestAvailableDate', '')
    
    # Initialize data dictionary
    data = {}
    
    # Process all ratio elements
    for ratio in root.findall('.//Ratio'):
        field_name = ratio.get('FieldName')
        if field_name:
            data[field_name] = parse_xml_value(ratio)
    
    # Process forecast data
    forecast_data = root.find('.//ForecastData')
    forecast = None
    if forecast_data is not None:
        try:
            forecast = ForecastMetrics(
                consensus_recommendation=get_ratio_value(data, 'ConsRecom'),
                target_price=get_ratio_value(data, 'TargetPrice'),
                projected_growth_rate=get_ratio_value(data, 'ProjIGrowthRate'),
                projected_pe=get_ratio_value(data, 'ProjPE'),
                projected_sales=get_ratio_value(data, 'ProjSales'),
                projected_sales_growth=get_ratio_value(data, 'ProjSalesQ'),
                projected_eps=get_ratio_value(data, 'ProjEPS'),
                projected_eps_q=get_ratio_value(data, 'ProjEPSQ'),
                projected_profit=get_ratio_value(data, 'ProjProfit'),
                projected_operating_margin=get_ratio_value(data, 'ProjOPS')
            )
        except Exception as e:
            print(f"Warning: Could not parse forecast data: {e}")
            forecast = None
    
    stock_info = StockInfo(
        # Metadata
        latest_available_date=latest_date,
        price_currency=price_currency,
        reporting_currency=reporting_currency,
        exchange_rate=exchange_rate,
        
        # Price and Volume
        current_price=get_ratio_value(data, 'NPRICE'),
        high_52week=get_ratio_value(data, 'NHIG'),
        low_52week=get_ratio_value(data, 'NLOW'),
        pricing_date=data.get('PDATE', ''),
        volume_10day_avg=get_ratio_value(data, 'VOL10DAVG'),
        enterprise_value=get_ratio_value(data, 'EV'),
        
        # Income Statement
        market_cap=get_ratio_value(data, 'MKTCAP'),
        revenue_ttm=get_ratio_value(data, 'TTMREV'),
        ebitda=get_ratio_value(data, 'TTMEBITD'),
        net_income_ttm=get_ratio_value(data, 'TTMNIAC'),
        
        # Per Share
        eps_ttm=get_ratio_value(data, 'TTMEPSXCLX'),
        revenue_per_share=get_ratio_value(data, 'TTMREVPS'),
        book_value_per_share=get_ratio_value(data, 'QBVPS'),
        cash_per_share=get_ratio_value(data, 'QCSHPS'),
        cash_flow_per_share=get_ratio_value(data, 'TTMCFSHR'),
        dividend_per_share=get_ratio_value(data, 'TTMDIVSHR'),
        
        # Margins
        gross_margin=get_ratio_value(data, 'TTMGROSMGN'),
        operating_margin=get_ratio_value(data, 'TTMOPMGN'),
        net_profit_margin=get_ratio_value(data, 'TTMNPMGN'),
        
        # Growth
        revenue_growth_rate=get_ratio_value(data, 'TTMREVCHG'),
        eps_growth_rate=get_ratio_value(data, 'TTMEPSCHG'),
        
        # Valuation
        pe_ratio=get_ratio_value(data, 'PEEXCLXOR'),
        price_to_book=get_ratio_value(data, 'PRICE2BK'),
        price_to_sales=get_ratio_value(data, 'TMPR2REV'),
        
        # Company Info
        employee_count=int(get_ratio_value(data, 'Employees')),
        
        # Forecast
        forecast=forecast
    )
    
    # Compute additional metrics
    stock_info.compute_derived_metrics(current_volume)
    
    return stock_info

In [5]:
from ib_insync import IB, Stock
import xml.etree.ElementTree as ET

# # Connect to IB
# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=1)

try:
    stock_info = get_stock_info(ib, 'AAPL', current_volume=1000000)
    print(f"Current Price: ${stock_info.current_price:.2f}")
    print(f"52-week Range: ${stock_info.low_52week:.2f} - ${stock_info.high_52week:.2f}")
    print(f"Volume vs 10-day avg: {stock_info.volume_vs_10day_avg_pct:.2f}%")
except Exception as e:
    print(f"Error getting stock info: {e}")
finally:
    ib.disconnect()

Current Price: $228.28
52-week Range: $164.07 - $237.49
Volume vs 10-day avg: 2273954.49%


In [8]:

stock_info.__dict__

{'latest_available_date': '2024-09-28',
 'price_currency': 'USD',
 'reporting_currency': 'USD',
 'exchange_rate': 1.0,
 'current_price': 228.28,
 'high_52week': 237.49,
 'low_52week': 164.075,
 'pricing_date': '2024-11-19T00:00:00',
 'volume_10day_avg': 43.97432,
 'enterprise_value': 3492098.0,
 'market_cap': 3450640.0,
 'revenue_ttm': 391035.0,
 'ebitda': 134661.0,
 'net_income_ttm': 93736.0,
 'eps_ttm': 6.06992,
 'revenue_per_share': 25.37854,
 'book_value_per_share': 3.76734,
 'cash_per_share': 4.31117,
 'cash_flow_per_share': 6.82635,
 'dividend_per_share': 0.98,
 'gross_margin': 46.20635,
 'operating_margin': 0.0,
 'net_profit_margin': 0.0,
 'revenue_growth_rate': 0.0,
 'eps_growth_rate': 0.0,
 'pe_ratio': 37.6084,
 'price_to_book': 60.59456,
 'price_to_sales': 0.0,
 'employee_count': 164000,
 'forecast': ForecastMetrics(consensus_recommendation=2.1429, target_price=209.1882, projected_growth_rate=0.0, projected_pe=34.62565, projected_sales=386988.0146, projected_sales_growth=9286

In [17]:
TSLA_info = get_stock_info(ib, ticker='TSLA')
TSLA_info

<class 'str'>
<?xml version="1.0" encoding="UTF-8"?>
<ReportSnapshot Major="1" Minor="0" Revision="1">
	<CoIDs>
		<CoID Type="RepNo">C8279</CoID>
		<CoID Type="CompanyName">Tesla Inc</CoID>
		<CoID Type="IRSNo">912197729</CoID>
		<CoID Type="CIKNo">0001318605</CoID>
		<CoID Type="OrganizationPermID">5088024644</CoID>
	</CoIDs>
	<Issues>
		<Issue ID="1" Type="C" Desc="Common Stock" Order="1">
			<IssueID Type="Name">Ordinary Shares</IssueID>
			<IssueID Type="Ticker">TSLA</IssueID>
			<IssueID Type="RIC">TSLA.O</IssueID>
			<IssueID Type="DisplayRIC">TSLA.OQ</IssueID>
			<IssueID Type="InstrumentPI">67910050</IssueID>
			<IssueID Type="QuotePI">72106022</IssueID>
			<Exchange Code="NASD" Country="USA">NASDAQ</Exchange>
			<GlobalListingType>OSR</GlobalListingType>
			<MostRecentSplit Date="2022-08-25">3.00003</MostRecentSplit>
		</Issue>
		<Issue ID="2" Type="P" Desc="Preferred Stock" Order="1">
			<IssueID Type="Name">Preference Shares Series B</IssueID>
			<IssueID Type="InstrumentPI"

StockInfo(tradingHours='20241120:0400-20241120:2000;20241121:0400-20241121:2000;20241122:0400-20241122:2000;20241123:CLOSED;20241124:CLOSED;20241125:0400-20241125:2000', liquidHours='20241120:0930-20241120:1600;20241121:0930-20241121:1600;20241122:0930-20241122:1600;20241123:CLOSED;20241124:CLOSED;20241125:0930-20241125:1600', segment='Consumer, Cyclical', industry='Consumer, Cyclical', category='Auto Manufacturers', revenue=0.0, netIncome=0.0, marketCap=0.0, sharesOutstanding=0, eps=0.0, pe=0.0, fpe=0.0, nextEarningsDate='')

In [18]:
TSLA_info.__dict__

{'tradingHours': '20241120:0400-20241120:2000;20241121:0400-20241121:2000;20241122:0400-20241122:2000;20241123:CLOSED;20241124:CLOSED;20241125:0400-20241125:2000',
 'liquidHours': '20241120:0930-20241120:1600;20241121:0930-20241121:1600;20241122:0930-20241122:1600;20241123:CLOSED;20241124:CLOSED;20241125:0930-20241125:1600',
 'segment': 'Consumer, Cyclical',
 'industry': 'Consumer, Cyclical',
 'category': 'Auto Manufacturers',
 'revenue': 0.0,
 'netIncome': 0.0,
 'marketCap': 0.0,
 'sharesOutstanding': 0,
 'eps': 0.0,
 'pe': 0.0,
 'fpe': 0.0,
 'nextEarningsDate': ''}

In [54]:
scanData[0].contractDetails.contract
scanData[0].contractDetails.__dict__
scanData[3].rank

3

In [7]:



symbol = scanData[0].contractDetails.contract.symbol


ss = Stock(symbol)
cds = ib.reqContractDetails(ss)
cds

[ContractDetails(contract=Contract(secType='STK', conId=13824, symbol='WMT', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='WMT', tradingClass='WMT'), marketName='WMT', minTick=0.01, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,CASHQTY,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,DUR,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,PEGMID,POSTATS,POSTONLY,PREOPGRTH,PRICECHK,REL,REL2MID,RELPCTOFS,RPI,RTH,RTHIGNOPG,SCALE,SCALEODD,SCALERST,SIZECHK,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,BYX,IEX,EDGX,FOXRIVER,PEARL,NYSENAT,LTSE,MEMX,IBEOS,OVERNIGHT,TPLUS0,PSX', priceMagnifier=1, underConId=0, longName='WALMART INC', contractMonth='', industry='Consumer, Cyclical', category='Retail', subcategory='Retail-Disco

In [46]:
cds = stockbot.scan_results[0][0]


cds.contractDetails.contract.__dict__

{'secType': 'STK',
 'conId': 727520246,
 'symbol': 'PRFX',
 'lastTradeDateOrContractMonth': '',
 'strike': 0.0,
 'right': '',
 'multiplier': '',
 'exchange': 'SMART',
 'primaryExchange': '',
 'currency': 'USD',
 'localSymbol': 'PRFX',
 'tradingClass': 'SCM',
 'includeExpired': False,
 'secIdType': '',
 'secId': '',
 'comboLegsDescrip': '',
 'comboLegs': [],
 'deltaNeutralContract': None}

In [18]:
nasdaq_data.iloc[14]['Value']

' BRIEF: For the 39 weeks ended 29 June 2024, Apple Inc revenues increased 1% to $296.11B. Net income increased 7% to $79B. Revenues reflect Europe segment increase of 6% to $76.4B, Americas segment increase of 2% to $125.38B, Japan segment increase of 2% to $19.13B. Net income benefited from Americas segment income increase of 13% to $50.64B, Europe segment income increase of 16% to $31.87B, Rest of Asia Pacific segment income increase of 6% to $10B.'

In [16]:
from dataclasses import dataclass
from typing import Optional, Dict, Any, List, Union
from datetime import datetime
import pandas as pd

@dataclass
class FinancialMetrics:
    eps: float
    dividend_per_share: float
    total_revenue: float
    dividend: float
    shares_out: float
    value: float

@dataclass
class CompanyInfo:
    industry: str
    exchange: str
    company_status: str
    global_listing_type: str
    symbol: str
    industry_index: str

class StockDataProcessor:
    INDUSTRY_INDICES = {
        'Holding Companies, Nec': '^GSPC',
        'Household Appliance Stores': '^IXIC',
        'Technology': '^IXIC',
        'Financial': '^BKX',
        'Healthcare': '^RXI',
        'Energy': '^XOI',
        'Consumer Discretionary': '^RLX',
        'Industrial': '^XLI'
    }

    def __init__(self):
        self.raw_data = None
        self.processed_data = None
        
    def process_stock_data(self, data: Union[pd.DataFrame, List[Dict], Dict[str, Any]]) -> Dict[str, Any]:
        """
        Process stock data from multiple input formats
        
        Args:
            data: Can be one of:
                - pandas DataFrame from explore_fundamental_data
                - List of dictionaries with Report/Metric/Value structure
                - Dictionary with metric-value pairs
        """
        if isinstance(data, pd.DataFrame):
            self.raw_data = self._convert_df_to_dict(data)
        elif isinstance(data, list):
            self.raw_data = self._convert_list_to_dict(data)
        elif isinstance(data, dict):
            self.raw_data = data
        else:
            raise ValueError("Unsupported data format. Please provide DataFrame, List[Dict], or Dict")
            
        processed_data = {
            'symbol': self._extract_symbol(),
            'financial_metrics': self._extract_financial_metrics(self.raw_data),
            'company_info': self._extract_company_info(self.raw_data),
            'analysis_metrics': self._calculate_analysis_metrics(self.raw_data),
            'timestamp': datetime.now().isoformat()
        }
        
        self.processed_data = processed_data
        return processed_data
    
    def _convert_df_to_dict(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Convert DataFrame to dictionary format"""
        return {row['Metric']: row['Value'] for _, row in df.iterrows()}
    
    def _convert_list_to_dict(self, data_list: List[Dict]) -> Dict[str, Any]:
        """Convert list of dictionaries to single dictionary"""
        return {item['Metric']: item['Value'] for item in data_list}
    
    def _extract_symbol(self) -> str:
        """Extract symbol from data or use default"""
        # Try different possible locations for symbol
        text = str(self.raw_data.get('Text', ''))
        if 'Apple' in text:
            return 'AAPL'
        elif 'JPMorgan' in text:
            return 'JPM'
        return 'UNKNOWN'
    
    def _extract_financial_metrics(self, data: Dict) -> FinancialMetrics:
        """Extract and standardize financial metrics"""
        return FinancialMetrics(
            eps=float(data.get('EPS', 0.0)),
            dividend_per_share=float(data.get('DividendPerShare', 0.0)),
            total_revenue=float(data.get('TotalRevenue', 0.0)),
            dividend=float(data.get('Dividend', 0.0)),
            shares_out=float(data.get('SharesOut', 0.0)),
            value=float(data.get('Value', 0.0))
        )
    
    def _extract_company_info(self, data: Dict) -> CompanyInfo:
        """Extract and standardize company information"""
        industry = data.get('Industry', '')
        return CompanyInfo(
            industry=industry,
            exchange=data.get('Exchange', ''),
            company_status=data.get('CoStatus', ''),
            global_listing_type=data.get('GlobalListingType', ''),
            symbol=self._extract_symbol(),
            industry_index=self._get_industry_index(industry)
        )
    
    def _get_industry_index(self, industry: str) -> str:
        """Get the corresponding index symbol for an industry"""
        return self.INDUSTRY_INDICES.get(industry, '^GSPC')
    
    def _calculate_analysis_metrics(self, data: Dict) -> Dict[str, float]:
        """Calculate additional analysis metrics"""
        return {
            'market_cap': self._calculate_market_cap(data),
            'dividend_yield': self._calculate_dividend_yield(data),
            'pe_ratio': self._calculate_pe_ratio(data),
            'price_to_book': self._calculate_price_to_book(data)
        }
    
    def _calculate_market_cap(self, data: Dict) -> float:
        shares_out = float(data.get('SharesOut', 0))
        latest_price = float(data.get('Value', 0))
        return shares_out * latest_price
    
    def _calculate_dividend_yield(self, data: Dict) -> float:
        dividend = float(data.get('DividendPerShare', 0))
        price = float(data.get('Value', 0))
        return (dividend / price * 100) if price > 0 else 0
    
    def _calculate_pe_ratio(self, data: Dict) -> float:
        eps = float(data.get('EPS', 0))
        price = float(data.get('Value', 0))
        return price / eps if eps > 0 else 0
    
    def _calculate_price_to_book(self, data: Dict) -> float:
        book_value = float(data.get('BookValue', 1))
        price = float(data.get('Value', 0))
        return price / book_value if book_value > 0 else 0
    
    def to_dataframe(self) -> pd.DataFrame:
        """Convert processed data to pandas DataFrame"""
        if not self.processed_data:
            raise ValueError("No data has been processed yet")
            
        flat_data = {
            'Symbol': self.processed_data['symbol'],
            'Industry': self.processed_data['company_info'].industry,
            'Industry_Index': self.processed_data['company_info'].industry_index,
            'Exchange': self.processed_data['company_info'].exchange,
            'EPS': self.processed_data['financial_metrics'].eps,
            'Dividend': self.processed_data['financial_metrics'].dividend,
            'Total_Revenue': self.processed_data['financial_metrics'].total_revenue,
            'Market_Cap': self.processed_data['analysis_metrics']['market_cap'],
            'Dividend_Yield': self.processed_data['analysis_metrics']['dividend_yield'],
            'PE_Ratio': self.processed_data['analysis_metrics']['pe_ratio'],
            'Price_to_Book': self.processed_data['analysis_metrics']['price_to_book'],
            'Timestamp': self.processed_data['timestamp']
        }
        
        return pd.DataFrame([flat_data])

# Example usage:
def process_stock_fundamentals(raw_data: List[Dict]) -> pd.DataFrame:
    processor = StockDataProcessor()
    processor.process_stock_data(raw_data)
    return processor.to_dataframe()

raw_data = nyse_data
raw_data = nasdaq_data

# Example usage:
# nasdaq_data = explore_fundamental_data(ib, 'AAPL', 'NASDAQ')
processor = StockDataProcessor()
processed_data = processor.process_stock_data(nasdaq_data)
result_df = processor.to_dataframe()
result_df

,Symbol,Industry,Industry_Index,Exchange,EPS,Dividend,Total_Revenue,Market_Cap,Dividend_Yield,PE_Ratio,Price_to_Book,Timestamp
0,AAPL,Household Appliance Stores,^IXIC,Not Available,0.98,0.1575,2.391760e+11,1.499641e+10,61.989719,1.012347,0.9921,2024-11-16T16:11:25.505771


In [4]:
import requests

def get_market_movers(api_key, direction=None, outputsize=30, country='USA', price_greater_than=None, dp=5):
    base_url = "https://api.twelvedata.com/market_movers/stocks"
    params = {
        'apikey': api_key,
        'direction': direction,
        'outputsize': outputsize,
        'country': country,
        'price_greater_than': price_greater_than,
        'dp': dp
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage
api_key = '171136ac7161454b8f4abeb987c72b02'
market_movers = get_market_movers(api_key)
display(market_movers)

{'code': 403,
 'message': '/market_movers/stocks is available exclusively with pro or enterprise plans. Consider upgrading your API Key now at https://twelvedata.com/pricing',
 'status': 'error'}

# Use IB Scanner

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

In [2]:
import scanner.scanner as scanner

# Usage example for gaps up
scanner_up = scanner.StockScanner(
    ib=ib,
    price_range=(1, 50),
    volume_range=(50_000, float('inf')),
    gap_range=(4, 20)  # positive range for up gaps
)

# Usage example for gaps down
# scanner_down = sb_scanner.StockScanner(
#     ib=ib,
#     price_range=(1, 50),
#     volume_range=(50_000, float('inf')),
#     gap_range=(-20, -4)  # negative range for down gaps
# )
        
results = scanner_up.run_scanner()
results

Starting scan for UP gaps using most recent trading day data...
Found 50 total stocks to analyze...
Processing batch 1/3
Processing batch 2/3
Processing batch 3/3

Results saved to stock_scan_results_20241116_104032.csv

Found 12 matches meeting all criteria:
   Symbol  Current Price  Volume  Gap %        Date
0    ATAI           1.61   56459  17.52  2024-11-15
1    RCAT           4.70   64981  16.92  2024-11-15
2    LABD           7.08  350524  16.07  2024-11-15
3    UVIX           3.87  587120  13.82  2024-11-15
4    PTLE           3.25  104006  13.24  2024-11-15
5     CAN           1.65  153369  11.49  2024-11-15
6    IONQ          29.14  309979  11.39  2024-11-15
7    UVXY          22.10  237440  10.56  2024-11-15
8    SOXS          25.13  395256   9.98  2024-11-15
9    GORV           1.13   72524   9.71  2024-11-15
10   RKLB          19.00  362101   9.45  2024-11-15
11    HUT          25.26   60692   9.16  2024-11-15


Error 162, reqId 81: Historical Market Data Service error message:API scanner subscription cancelled: 81


,Symbol,Exchange,Current Price,Previous Close,Volume,Gap %,High,Low,Date,Previous Date
0,ATAI,SMART,1.61,1.37,56459,17.52,1.65,1.43,2024-11-15,2024-11-14
1,RCAT,SMART,4.70,4.02,64981,16.92,5.01,4.42,2024-11-15,2024-11-14
2,LABD,SMART,7.08,6.10,350524,16.07,7.12,6.13,2024-11-15,2024-11-14
3,UVIX,SMART,3.87,3.40,587120,13.82,4.18,3.46,2024-11-15,2024-11-14
4,PTLE,SMART,3.25,2.87,104006,13.24,4.79,2.79,2024-11-15,2024-11-14
5,CAN,SMART,1.65,1.48,153369,11.49,1.68,1.43,2024-11-15,2024-11-14
6,IONQ,SMART,29.14,26.16,309979,11.39,29.48,25.03,2024-11-15,2024-11-14
7,UVXY,SMART,22.10,19.99,237440,10.56,23.49,20.28,2024-11-15,2024-11-14
8,SOXS,SMART,25.13,22.85,395256,9.98,25.33,23.71,2024-11-15,2024-11-14
9,GORV,SMART,1.13,1.03,72524,9.71,1.56,0.96,2024-11-15,2024-11-14


In [ ]:
"https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/scanners.ipynb"

sub = ScannerSubscription(
    instrument='STK', 
    locationCode='STK.US.MAJOR', 
    scanCode='TOP_PERC_GAIN')

tagValues = [
    TagValue("changePercAbove", "20"),
    TagValue('priceAbove', 5),
    TagValue('priceBelow', 50)]

scanData = ib.reqScannerData(sub, [], tagValues)

print(f'{len(scanData)} results, first one:')
print(scanData[0])

8 results, first one:
ScanData(rank=0, contractDetails=ContractDetails(contract=Contract(secType='STK', conId=229152096, symbol='CDXC', exchange='SMART', currency='USD', localSymbol='CDXC', tradingClass='SCM'), marketName='SCM', minTick=0.0, orderTypes='', validExchanges='', priceMagnifier=0, underConId=0, longName='', contractMonth='', industry='', category='', subcategory='', timeZoneId='', tradingHours='', liquidHours='', evRule='', evMultiplier=0, mdSizeMultiplier=1, aggGroup=0, underSymbol='', underSecType='', marketRuleIds='', secIdList=[], realExpirationDate='', lastTradeTime='', stockType='', minSize=0.0, sizeIncrement=0.0, suggestedSizeIncrement=0.0, cusip='', ratings='', descAppend='', bondType='', couponType='', callable=False, putable=False, coupon=0, convertible=False, maturity='', issueDate='', nextOptionDate='', nextOptionType='', nextOptionPartial=False, notes=''), distance='', benchmark='', projection='', legsStr='')


In [ ]:
import pandas as pd

# Assuming scanData is a list of ScanData objects
scan_data_list = []

for data in scanData:
    contract = data.contractDetails.contract
    scan_data_list.append({
        'rank': data.rank,
        'symbol': contract.symbol,
        'exchange': contract.exchange,
        'currency': contract.currency,
        'localSymbol': contract.localSymbol,
        'tradingClass': contract.tradingClass,
        'marketName': data.contractDetails.marketName,
        'minTick': data.contractDetails.minTick,
        'longName': data.contractDetails.longName,
        'industry': data.contractDetails.industry,
        'category': data.contractDetails.category,
        'subcategory': data.contractDetails.subcategory,
        'timeZoneId': data.contractDetails.timeZoneId,
        'tradingHours': data.contractDetails.tradingHours,
        'liquidHours': data.contractDetails.liquidHours,
        'evRule': data.contractDetails.evRule,
        'evMultiplier': data.contractDetails.evMultiplier,
        'mdSizeMultiplier': data.contractDetails.mdSizeMultiplier,
        'aggGroup': data.contractDetails.aggGroup,
        'underSymbol': data.contractDetails.underSymbol,
        'underSecType': data.contractDetails.underSecType,
        'marketRuleIds': data.contractDetails.marketRuleIds,
        'realExpirationDate': data.contractDetails.realExpirationDate,
        'lastTradeTime': data.contractDetails.lastTradeTime,
        'stockType': data.contractDetails.stockType,
        'minSize': data.contractDetails.minSize,
        'sizeIncrement': data.contractDetails.sizeIncrement,
        'suggestedSizeIncrement': data.contractDetails.suggestedSizeIncrement,
        'cusip': data.contractDetails.cusip,
        'ratings': data.contractDetails.ratings,
        'descAppend': data.contractDetails.descAppend,
        'bondType': data.contractDetails.bondType,
        'couponType': data.contractDetails.couponType,
        'callable': data.contractDetails.callable,
        'putable': data.contractDetails.putable,
        'coupon': data.contractDetails.coupon,
        'convertible': data.contractDetails.convertible,
        'maturity': data.contractDetails.maturity,
        'issueDate': data.contractDetails.issueDate,
        'nextOptionDate': data.contractDetails.nextOptionDate,
        'nextOptionType': data.contractDetails.nextOptionType,
        'nextOptionPartial': data.contractDetails.nextOptionPartial,
        'notes': data.contractDetails.notes,
        'distance': data.distance,
        'benchmark': data.benchmark,
        'projection': data.projection,
        'legsStr': data.legsStr
    })

# show all columns
pd.set_option('display.max_columns', None)
df = pd.DataFrame(scan_data_list).set_index('rank') # ['symbol','exchange', 'currency']
display(df)

,symbol,exchange,currency,localSymbol,tradingClass,marketName,minTick,longName,industry,category,subcategory,timeZoneId,tradingHours,liquidHours,evRule,evMultiplier,mdSizeMultiplier,aggGroup,underSymbol,underSecType,marketRuleIds,realExpirationDate,lastTradeTime,stockType,minSize,sizeIncrement,suggestedSizeIncrement,cusip,ratings,descAppend,bondType,couponType,callable,putable,coupon,convertible,maturity,issueDate,nextOptionDate,nextOptionType,nextOptionPartial,notes,distance,benchmark,projection,legsStr
rank,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,CDXC,SMART,USD,CDXC,SCM,SCM,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,
1,PRLB,SMART,USD,PRLB,PRLB,PRLB,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,
2,TILE,SMART,USD,TILE,NMS,NMS,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,
3,VIR,SMART,USD,VIR,NMS,NMS,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,
4,MD,SMART,USD,MD,MD,MD,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,
5,RGC,SMART,USD,RGC,SCM,SCM,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,
6,CCTS,SMART,USD,CCTS,NMS,NMS,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,
7,QMMM,SMART,USD,QMMM,SCM,SCM,0.0,,,,,,,,,0,1,0,,,,,,,0.0,0.0,0.0,,,,,,False,False,0,False,,,,,False,,,,,


In [ ]:
sub.abovePrice = 200
scanData = ib.reqScannerData(sub)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

['MDGL', 'WAT', 'TEAM', 'CHTR', 'POWL', 'CVCO', 'LULU', 'IESC', 'CABO', 'FSLR', 'MCK', 'TFX', 'LECO', 'MTD', 'FNGU', 'WING', 'WINA', 'KAI', 'SYK', 'DPZ', 'BIO', 'WST', 'ALNY', 'COR', 'IDXX', 'ULTA', 'WDAY', 'CDNS', 'XSD', 'TMO', 'UFPT', 'CRWD', 'FI', 'ELV', 'ANET', 'RTH', 'LFUS', 'PEN', 'MNDY', 'RGA', 'AYI', 'VEEV', 'HUBS', 'INTU', 'UI', 'EVR', 'RBC', 'PODD', 'NXPI', 'DJCO']


In [ ]:
import scanner 

# Create a scanner subscription

sc = scanner.Scanner(ib, 50, 'LONG')

NameError: name 'ib' is not defined

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Union, Tuple, Optional
import pandas as pd
from ib_insync import IB, Stock

@dataclass
class ForecastMetrics:
    consensus_recommendation: float
    target_price: float
    projected_growth_rate: float
    projected_pe: float
    projected_sales: float
    projected_sales_growth: float
    projected_eps: float
    projected_eps_q: float
    projected_profit: float
    projected_operating_margin: float

@dataclass
class StockInfo:
    latest_available_date: str
    price_currency: str
    reporting_currency: str
    exchange_rate: float
    current_price: float
    high_52week: float
    low_52week: float
    pricing_date: str
    volume_10day_avg: float
    enterprise_value: float
    market_cap: float
    revenue_ttm: float
    ebitda: float
    net_income_ttm: float
    eps_ttm: float
    revenue_per_share: float
    book_value_per_share: float
    cash_per_share: float
    cash_flow_per_share: float
    dividend_per_share: float
    gross_margin: float
    operating_margin: float
    net_profit_margin: float
    revenue_growth_rate: float
    eps_growth_rate: float
    pe_ratio: float
    price_to_book: float
    price_to_sales: float
    employee_count: int
    forecast: ForecastMetrics
    price_to_10day_avg: float
    volume_vs_10day_avg_pct: float
    distance_from_52wk_high_pct: float
    distance_from_52wk_low_pct: float

class StockAnalyzer:
    def __init__(self):
        self.ib = IB()
        self.stock_data: Dict[str, StockInfo] = {}
        
    def connect(self, host: str = '127.0.0.1', port: int = 7497, client_id: int = 1):
        """Connect to Interactive Brokers"""
        try:
            self.ib.connect(host, port, clientId=client_id)
            return True
        except Exception as e:
            print(f"Connection error: {e}")
            return False

    def disconnect(self):
        """Disconnect from Interactive Brokers"""
        if self.ib.isConnected():
            self.ib.disconnect()

    def get_fundamentals(self, symbols: List[str]) -> Dict[str, StockInfo]:
        """
        Fetch fundamental data for multiple stock symbols
        
        Args:
            symbols: List of stock ticker symbols
            
        Returns:
            Dictionary mapping symbols to their fundamental data
        """
        try:
            for symbol in symbols:
                stock_info = self._get_stock_info(symbol)
                self.stock_data[symbol] = stock_info
            return self.stock_data
        except Exception as e:
            print(f"Error fetching fundamentals: {e}")
            return {}

    def _get_stock_info(self, symbol: str) -> StockInfo:
        """Internal method to get stock info for a single symbol"""
        # This would call your existing get_stock_info function
        return fundamentals.get_stock_info(self.ib, symbol)

    def to_dataframe(self) -> pd.DataFrame:
        """
        Convert all stored stock data to a pandas DataFrame
        
        Returns:
            DataFrame with stock fundamentals
        """
        if not self.stock_data:
            return pd.DataFrame()

        # Convert the nested data structure to a flat dictionary
        flat_data = []
        for symbol, info in self.stock_data.items():
            data_dict = info.__dict__.copy()
            forecast_dict = data_dict.pop('forecast').__dict__
            
            # Prefix forecast metrics with 'forecast_'
            forecast_dict = {f'forecast_{k}': v for k, v in forecast_dict.items()}
            
            data_dict.update(forecast_dict)
            data_dict['symbol'] = symbol
            flat_data.append(data_dict)

        return pd.DataFrame(flat_data).set_index('symbol')

    def filter_stocks(self, criteria: Dict[str, Union[Tuple[float, float], float]]) -> pd.DataFrame:
        """
        Filter stocks based on specified criteria
        
        Args:
            criteria: Dictionary of metrics and their target values/ranges
                     For ranges, use tuple (min_value, max_value)
                     For exact values, use a single float
        
        Returns:
            Filtered DataFrame meeting all criteria
        
        Example:
            criteria = {
                'pe_ratio': (0, 15),
                'market_cap': (1e9, 1e11),
                'dividend_per_share': (0.5, None)  # Greater than 0.5
            }
        """
        df = self.to_dataframe()
        if df.empty:
            return df

        for metric, value in criteria.items():
            if metric not in df.columns:
                print(f"Warning: Metric '{metric}' not found in data")
                continue

            if isinstance(value, tuple):
                min_val, max_val = value
                if min_val is not None:
                    df = df[df[metric] >= min_val]
                if max_val is not None:
                    df = df[df[metric] <= max_val]
            else:
                df = df[df[metric] == value]

        return df

    def get_summary_metrics(self, symbols: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Get key summary metrics for specified symbols or all symbols
        
        Args:
            symbols: Optional list of symbols to summarize. If None, summarize all
            
        Returns:
            DataFrame with key metrics
        """
        df = self.to_dataframe()
        if df.empty:
            return df

        if symbols:
            df = df.loc[symbols]

        key_metrics = [
            'current_price', 'pe_ratio', 'market_cap', 'dividend_per_share',
            'eps_ttm', 'revenue_ttm', 'gross_margin', 'net_profit_margin',
            'forecast_target_price', 'forecast_consensus_recommendation'
        ]

        return df[key_metrics]

    def __enter__(self):
        """Context manager support"""
        self.connect()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager support"""
        self.disconnect()